In [1]:
import tensorflow as tf
import numpy
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits
co = {}; _add_supported_quantized_objects(co)

model_path = f"/eos/project/c/cms-l1ml/public/Pawel/axol1tl_model.h5"

model = tf.keras.models.load_model(model_path, custom_objects=co)
#model_config = model.get_config()

2025-08-15 10:46:16.570214: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-15 10:46:16.608278: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-15 10:46:16.608351: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-15 10:46:16.608403: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-15 10:46:16.617247: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-15 10:46:16.617977: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
data_path = f"/eos/project/c/cms-l1ml/public/Pawel/complete.h5"
data_file = h5py.File(data_path, 'r')

Copying weights into the model

In [3]:
model.compile() #############################

In [4]:
ds_weights = data_file['model']['trimmed_encoder']['model_weights']
for layer in iter(model.layers):
    #print(layer.name)
    if layer.name in ('input_1','subtract','dot'): continue

    if layer.name == 'q_dense_4':
        weights_group = ds_weights['q_dense_4']['variational_auto_encoder']['vae__encoder']['q_dense_4']
        layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))
        continue

    weights_group = ds_weights[layer.name][layer.name]
    layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))

Background prediction (test)

In [11]:
X = data_file['data']["Background_data"]["Test"]["DATA"][:]
scale_data = data_file['data']['Normalisation']['norm_scale'][:].flatten()
offset_data =  data_file['data']['Normalisation']['norm_bias'][:].flatten()

X_flat = X.reshape(X.shape[0], -1)
X_int = (X_flat * scale_data) + offset_data
X_flat_int = X_int.astype('int')

In [6]:
quantizer = quantized_bits(bits=14, integer=7)
tf.executing_eagerly()

True

In [12]:
from tqdm import tqdm
batch_size = 128
predictions = []

for start in tqdm(range(0, 1024, batch_size), desc="Qkeras preds"):
        end = min(start + batch_size, X_flat_int.shape[0])
        X_for_qk = X_flat_int[start:end].astype('float')
        X_biased = quantizer(X_for_qk - offset_data).numpy().astype('float')
        X_scaled = quantizer(X_biased / scale_data)
        X_scaled_quantized_2 = quantizer(X_scaled)
        
        pred = model.predict(X_scaled_quantized_2)
        quantized_pred = quantizer(pred)
        predictions.append(quantized_pred)

Qkeras preds:   0%|          | 0/8 [00:00<?, ?it/s]

4/4 [==============================] - 0s 2ms/step


Qkeras preds:  25%|██▌       | 2/8 [00:00<00:00, 15.39it/s]

4/4 [==============================] - 0s 2ms/step


Qkeras preds:  50%|█████     | 4/8 [00:00<00:00, 16.41it/s]

4/4 [==============================] - 0s 2ms/step


Qkeras preds:  75%|███████▌  | 6/8 [00:00<00:00, 16.57it/s]

4/4 [==============================] - 0s 2ms/step


Qkeras preds: 100%|██████████| 8/8 [00:00<00:00, 16.58it/s]


In [13]:
predictions[0]

<tf.Tensor: shape=(128, 1), dtype=float32, numpy=
array([[ 4.09375 ],
       [ 0.953125],
       [ 1.734375],
       [ 0.921875],
       [ 3.359375],
       [ 3.3125  ],
       [ 3.125   ],
       [ 2.21875 ],
       [ 1.046875],
       [ 2.515625],
       [ 1.140625],
       [ 0.34375 ],
       [ 4.03125 ],
       [ 1.015625],
       [ 9.40625 ],
       [ 1.078125],
       [ 2.546875],
       [ 1.125   ],
       [ 0.78125 ],
       [ 1.671875],
       [ 1.421875],
       [ 2.46875 ],
       [ 3.21875 ],
       [ 1.40625 ],
       [ 1.8125  ],
       [ 3.15625 ],
       [ 2.9375  ],
       [ 1.515625],
       [ 1.6875  ],
       [ 2.21875 ],
       [ 1.1875  ],
       [ 1.0625  ],
       [ 1.046875],
       [ 1.03125 ],
       [ 1.1875  ],
       [ 1.25    ],
       [ 1.1875  ],
       [ 2.234375],
       [ 2.03125 ],
       [ 1.734375],
       [ 1.53125 ],
       [ 0.984375],
       [ 2.609375],
       [ 1.5     ],
       [ 1.234375],
       [ 1.390625],
       [ 3.5625  ],
       [ 1

Signal prediction

In [7]:
X = data_file['data']["Signal_data"]["GluGluHToBB_M-125"]["DATA"][:]
X_flat = X.reshape(X.shape[0], -1)
X_int = (X_flat * scale_data) + offset_data
X_flat_int = X_int.astype('int')

In [10]:
signal_predictions = numpy.array(predictions)

In [14]:
signal_predictions[0]

array([[127.984375],
       [107.5     ],
       [127.984375],
       [127.984375],
       [127.984375],
       [127.984375],
       [111.921875],
       [105.609375],
       [110.625   ],
       [127.984375],
       [127.984375],
       [127.984375],
       [127.984375],
       [112.9375  ],
       [127.984375],
       [127.984375],
       [107.453125],
       [127.984375],
       [127.984375],
       [127.984375],
       [ 81.125   ],
       [127.984375],
       [127.984375],
       [127.984375],
       [ 57.359375],
       [ 36.3125  ],
       [127.984375],
       [127.984375],
       [127.984375],
       [127.984375],
       [107.953125],
       [127.984375],
       [ 49.875   ],
       [ 66.921875],
       [127.984375],
       [127.984375],
       [ 33.8125  ],
       [ 43.140625],
       [ 89.96875 ],
       [127.984375],
       [127.984375],
       [ 74.421875],
       [ 35.5625  ],
       [ 74.96875 ],
       [127.984375],
       [127.984375],
       [127.984375],
       [120.4

Conversion into a hls4ml model

In [ ]:
import hls4ml

hls_config = hls4ml.utils.config_from_keras_model(
    model,
    granularity='name', 
    default_precision='fixed<14,7>',
    backend='Vitis')
print("-----------------------------------")


In [ ]:
hls_model = hls4ml.converters.convert_from_keras_model(
    model, hls_config=hls_config, backend='Vitis',
    output_dir='./hls_test'
)

In [ ]:
hls_model.compile()

In [ ]:
X_test = numpy.ascontiguousarray(X_test)
y_hls = hls_model.predict(X_test)


In [ ]:
from tqdm import tqdm
batch_size = 128
predictions_hls = []

for start in tqdm(range(0, 1024, batch_size), desc="hls preds"):
        end = min(start + batch_size, X_flat_int.shape[0])
        X_for_qk = X_flat_int[start:end].astype('float')
        X_biased = quantizer(X_for_qk - offset_data).numpy().astype('float')
        X_scaled = quantizer(X_biased / scale_data)
        X_scaled_quantized_2 = quantizer(X_scaled)
        
        pred = hls_model.predict(numpy.ascontiguousarray(X_scaled_quantized_2))
        quantized_pred = quantizer(pred)
        predictions_hls.append(quantized_pred)

In [ ]:
predictions[0][:,0].numpy()

In [ ]:
print(numpy.column_stack((predictions[0][:,0].numpy(),predictions_hls[0][:,0].numpy())))